# Langchain Démo

## Chargement du document

In [1]:
from langchain.document_loaders import PyPDFLoader

In [2]:
loader = PyPDFLoader("uploads/code_penal.pdf")
code_penal = loader.load_and_split()

Première page du code pénal : 

In [3]:
code_penal[0]

Document(page_content="Code pénal - Dernière modification le 01 avril 2023 - Document généré le 15 avril 2023Code pénal\nPartie législative\nLivre Ier : Dispositions générales\nTitre Ier : De la loi pénale\nChapitre Ier : Des principes généraux\nArticle 111-1\n \nLes infractions pénales sont classées, suivant leur gravité, en crimes, délits et contraventions.\n \n \nArticle 111-2\n \nLa loi détermine les crimes et délits et fixe les peines applicables à leurs auteurs.\n \n \nLe règlement détermine les contraventions et fixe, dans les limites et selon les distinctions établies par la loi,\nles peines applicables aux contrevenants.\n \nArticle 111-3\n \nNul ne peut être puni pour un crime ou pour un délit dont les éléments ne sont pas définis par la loi, ou pour\nune contravention dont les éléments ne sont pas définis par le règlement.\n \n \nNul ne peut être puni d'une peine qui n'est pas prévue par la loi, si l'infraction est un crime ou un délit, ou par\nle règlement, si l'infraction 

## Découpage du document

In [4]:
from langchain.text_splitter import CharacterTextSplitter

In [5]:
article_splitter = CharacterTextSplitter(
    separator = "\nArticle",
    chunk_size = 1000,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
)

In [6]:
articles = article_splitter.split_documents(code_penal)

Created a chunk of size 1069, which is longer than the specified 1000
Created a chunk of size 1510, which is longer than the specified 1000
Created a chunk of size 1344, which is longer than the specified 1000
Created a chunk of size 1040, which is longer than the specified 1000
Created a chunk of size 1595, which is longer than the specified 1000
Created a chunk of size 1491, which is longer than the specified 1000
Created a chunk of size 3166, which is longer than the specified 1000
Created a chunk of size 1632, which is longer than the specified 1000
Created a chunk of size 1417, which is longer than the specified 1000
Created a chunk of size 1306, which is longer than the specified 1000
Created a chunk of size 1584, which is longer than the specified 1000
Created a chunk of size 2509, which is longer than the specified 1000
Created a chunk of size 1523, which is longer than the specified 1000
Created a chunk of size 1049, which is longer than the specified 1000
Created a chunk of s

Apercu d'un article :

In [7]:
articles[4]

Document(page_content="112-3\n \nLes lois relatives à la nature et aux cas d'ouverture des voies de recours ainsi qu'aux délais dans lesquels elles\ndoivent être exercées et à la qualité des personnes admises à se pourvoir sont applicables aux recours formés", metadata={'source': 'uploads/code_penal.pdf', 'page': 1})

Nombres approximatif d'articles :

In [8]:
len(articles)

1109

In [9]:
articles_page_content = [article.page_content for article in articles]

## Embedding du document

In [10]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


In [11]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [12]:
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [13]:
#articles_embedded = embeddings_model.embed_documents(articles_page_content)

## Vectorisation des articles

In [14]:
from langchain.vectorstores import Chroma

In [15]:
vector_db = Chroma.from_documents(articles,embeddings_model) 

## Requête à la bdd vectoriel

In [16]:
query = "Prison à perpétuité"
docs = vector_db.similarity_search(query, k=10)
for doc in docs:
    print(doc.page_content)
    print("------")

132-8
 
Lorsqu'une personne physique, déjà condamnée définitivement pour un crime ou pour un délit puni de dix
ans d'emprisonnement par la loi, commet un crime, le maximum de la peine de la réclusion criminelle ou de
la détention criminelle est la perpétuité si le maximum fixé par la loi pour ce crime est de vingt ou trente ans.
Le maximum de la peine est porté à trente ans de réclusion criminelle ou de détention criminelle si le crime
est puni de quinze ans.
------
422-2
 
La peine privative de liberté encourue par l'auteur ou le complice d'un acte de terrorisme est réduite de
moitié si, ayant averti les autorités administratives ou judiciaires, il a permis de faire cesser les agissements
incriminés ou d'éviter que l'infraction n'entraîne mort d'homme ou infirmité permanente et d'identifier, le cas
échéant, les autres coupables. Lorsque la peine encourue est la réclusion criminelle à perpétuité, celle-ci est
ramenée à vingt ans de réclusion criminelle.
------
221-2
 
Le meurtre qui pr

## Questions / Réponses avec le code pénal

### Prompt Template

In [17]:
from langchain.prompts import PromptTemplate

In [18]:
QA_PROMPT = PromptTemplate(
    template="""Voici des articles extraits du code pénal français : {text}
À partir de ces articles répond à la question suivante : {question}""",
    input_variables=["text", "question"]
)

### Chargement du LLM

In [19]:
from langchain.chat_models import ChatOpenAI

In [35]:
llm = ChatOpenAI(openai_api_key=openai_api_key,model="gpt-4",temperature=0)

### Création de la chaîne

In [36]:
from langchain.chains import RetrievalQA

In [37]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_db.as_retriever(),
    return_source_documents=True,
    chain_type="stuff",
)

### Utilisation

In [38]:
question = "Quels sont les conditions qui pourraient entraîner une condamnation à perpétuité?"
response = qa_chain({"query": question})
print(response["result"])
print("-------")
print(print(response["source_documents"][0].page_content))

Selon le Code pénal, une condamnation à perpétuité peut être prononcée dans les cas suivants :

1. Lorsqu'une personne physique, déjà condamnée définitivement pour un crime ou pour un délit puni de dix ans d'emprisonnement par la loi, commet un crime dont le maximum fixé par la loi est de vingt ou trente ans.

2. L'infraction prévue à l'article 224-1 est punie de la réclusion criminelle à perpétuité lorsqu'elle est précédée ou accompagnée de tortures ou d'actes de barbarie ou lorsqu'elle est suivie de la mort de la victime.

3. La provocation publique et directe, par tous moyens, à commettre un génocide est punie de la réclusion criminelle à perpétuité si cette provocation a été suivie d'effet.
-------
132-8
 
Lorsqu'une personne physique, déjà condamnée définitivement pour un crime ou pour un délit puni de dix
ans d'emprisonnement par la loi, commet un crime, le maximum de la peine de la réclusion criminelle ou de
la détention criminelle est la perpétuité si le maximum fixé par la loi